This notebook is used for performance comparison between my best model at the time of writing (Bayesian Ridge) and [Adil Shamim's model](https://www.kaggle.com/code/adilshamim8/predicting-the-beats-per-minute-of-songs-101). 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV, KFold
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import BayesianRidge, Ridge
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import StackingRegressor, RandomForestRegressor

import lightgbm as lgb
import xgboost as xgb
import catboost as cb

import os
import joblib

from src.feature_engineering import *
from src.modeling import *

import warnings
warnings.filterwarnings("ignore")

In [2]:
#config
DATA_PATH = "data/"
OUTPUT_PATH = "output/"
MODEL_PATH = "models/bayesian_alpha_1_1e_-06_alpha_2_0_01_lambda_1_0_01_lambda_2_0_01_trained.joblib"

In [3]:
df_train = pd.read_csv(DATA_PATH + "train.csv")
print(df_train.shape)
df_train.head(3)

(524164, 11)


,id,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy,BeatsPerMinute
0,0,0.603610,-7.636942,0.023500,0.000005,0.000001,0.051385,0.409866,290715.6450,0.826267,147.53020
1,1,0.639451,-16.267598,0.071520,0.444929,0.349414,0.170522,0.651010,164519.5174,0.145400,136.15963
2,2,0.514538,-15.953575,0.110715,0.173699,0.453814,0.029576,0.423865,174495.5667,0.624667,55.31989


In [4]:
df_test = pd.read_csv(DATA_PATH + "test.csv")
print(df_test.shape)
df_test.head(3)

(174722, 10)


,id,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy
0,524164,0.410013,-16.794967,0.023500,0.232910,0.012689,0.271585,0.664321,302901.5498,0.424867
1,524165,0.463071,-1.357000,0.141818,0.057725,0.257942,0.097624,0.829552,221995.6643,0.846000
2,524166,0.686569,-3.368928,0.167851,0.287823,0.210915,0.325909,0.304978,357724.0127,0.134067


Get current best model evaluation

In [5]:
model_ori = joblib.load(MODEL_PATH)
#BayesianRidge(alpha_1=1e-06, alpha_2=0.01, lambda_1=0.01, lambda_2=0.01)
model_ori

,steps,"[('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,max_iter,300
,tol,0.001
,alpha_1,1e-06
,alpha_2,0.01
,lambda_1,0.01
,lambda_2,0.01
,alpha_init,None


In [6]:
rmse_ori = cross_val_score(
    model_ori,
    df_train.drop(columns=['id', 'BeatsPerMinute']),
    df_train['BeatsPerMinute'],
    scoring="neg_mean_squared_error",
    cv=5,
).mean()
rmse_ori = (-rmse_ori)**0.5
rmse_ori

np.float64(26.46646924573802)

Best model + feature engineering

In [7]:
#Adil Shamim feature engineering
def engineer_features(df):
    """Create engineered features for the dataset
    
    Args:
        df: DataFrame containing the original features
        
    Returns:
        DataFrame with additional engineered features
    """
    # Create a copy to avoid modifying the original dataframe
    df_new = df.copy()
    
    # 1. Rhythm-based features - BPM is directly related to rhythm
    df_new['Rhythm_Energy'] = df_new['RhythmScore'] * df_new['Energy']
    df_new['Rhythm_Loudness'] = df_new['RhythmScore'] * df_new['AudioLoudness']
    
    # 2. Duration-related features - longer tracks might have different BPM patterns
    df_new['Duration_Minutes'] = df_new['TrackDurationMs'] / 60000  # Convert to minutes
    df_new['Duration_Energy_Ratio'] = df_new['TrackDurationMs'] / (df_new['Energy'] * 10000 + 1)  # Scaled for numerical stability
    
    # 3. Non-linear transformations - capture more complex relationships
    df_new['RhythmScore_Squared'] = df_new['RhythmScore'] ** 2
    df_new['Energy_Squared'] = df_new['Energy'] ** 2
    df_new['Log_Duration'] = np.log1p(df_new['TrackDurationMs'])  # log(1+x) to handle zeros
    
    # 4. Musical character features - representing the "feel" of the song
    df_new['Acoustic_Instrumental_Ratio'] = df_new['AcousticQuality'] / (df_new['InstrumentalScore'] + 0.01)  # Avoid division by zero
    df_new['Vocal_Energy'] = df_new['VocalContent'] * df_new['Energy']
    
    # 5. Performance and mood interactions
    df_new['Live_Energy'] = df_new['LivePerformanceLikelihood'] * df_new['Energy']
    df_new['Mood_Rhythm'] = df_new['MoodScore'] * df_new['RhythmScore']
    
    # 6. Composite metrics
    df_new['Audio_Intensity'] = (df_new['Energy'] * np.abs(df_new['AudioLoudness'])) / 10  # Scaled for better range
    df_new['Performance_Character'] = (df_new['LivePerformanceLikelihood'] + df_new['MoodScore']) / 2
    
    # 7. Ratios that might represent musical balance
    df_new['Energy_Loudness_Ratio'] = df_new['Energy'] / (np.abs(df_new['AudioLoudness']) + 0.01)
    df_new['Rhythm_Duration_Density'] = df_new['RhythmScore'] / df_new['Duration_Minutes']
    
    return df_new

In [8]:
df_train_temp = engineer_features(df_train)
print(df_train_temp.shape)
df_train_temp.head(3)

(524164, 26)


,id,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy,...,Energy_Squared,Log_Duration,Acoustic_Instrumental_Ratio,Vocal_Energy,Live_Energy,Mood_Rhythm,Audio_Intensity,Performance_Character,Energy_Loudness_Ratio,Rhythm_Duration_Density
0,0,0.603610,-7.636942,0.023500,0.000005,0.000001,0.051385,0.409866,290715.6450,0.826267,...,0.682717,12.580104,0.000536,0.019417,0.042458,0.247399,0.631015,0.230626,0.108052,0.124577
1,1,0.639451,-16.267598,0.071520,0.444929,0.349414,0.170522,0.651010,164519.5174,0.145400,...,0.021141,12.010791,1.237928,0.010399,0.024794,0.416289,0.236531,0.410766,0.008933,0.233207
2,2,0.514538,-15.953575,0.110715,0.173699,0.453814,0.029576,0.423865,174495.5667,0.624667,...,0.390208,12.069660,0.374500,0.069160,0.018475,0.218095,0.996567,0.226720,0.039131,0.176923


In [9]:
rmse_fe = cross_val_score(
    BayesianRidge(alpha_1=1e-06, alpha_2=0.01, lambda_1=0.01, lambda_2=0.01),
    df_train_temp.drop(columns=['id', 'BeatsPerMinute']),
    df_train_temp['BeatsPerMinute'],
    scoring="neg_mean_squared_error",
    cv=5,
).mean()
rmse_fe = (-rmse_fe)**0.5
rmse_fe

np.float64(26.466643184187255)

Best model + Robust Scaler

In [10]:
scaler = RobustScaler()

X_temp = df_train.drop(columns=['id', 'BeatsPerMinute'])
X_temp = pd.DataFrame(
    scaler.fit_transform(X_temp),
    columns=X_temp.columns
)

y_temp = df_train['BeatsPerMinute']

rmse_scaled = cross_val_score(
    BayesianRidge(alpha_1=1e-06, alpha_2=0.01, lambda_1=0.01, lambda_2=0.01),
    X_temp,
    y_temp,
    scoring="neg_mean_squared_error",
    cv=5,
).mean()
rmse_scaled = (-rmse_scaled)**0.5
rmse_scaled

np.float64(26.466297855116007)

FE + Robust Scaler + LightGBM/XGBoost/CatBoost

In [11]:
# Apply feature engineering and scaling
scaler = RobustScaler()

X_temp = engineer_features(df_train).drop(columns=['id', 'BeatsPerMinute'])
X_temp = pd.DataFrame(
    scaler.fit_transform(X_temp),
    columns=X_temp.columns
)

y_temp = df_train['BeatsPerMinute']

# Set up cross-validation strategy
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
folds = list(kf.split(X_temp))

# Initialize lists to store models and predictions
models = []
oof_predictions = np.zeros(len(X_temp))
rmse_list_lgb = []
rmse_list_xgb = []
rmse_list_cb = []
rmse_list_blend = []
rmse_list_lgb_clipped = []

# Train and evaluate models across folds
for fold_idx, (train_idx, val_idx) in enumerate(folds):
    print(f"\nTraining fold {fold_idx + 1}/{n_folds}")
    
    # Split data for this fold
    X_fold_train, X_fold_val = X_temp.iloc[train_idx], X_temp.iloc[val_idx]
    y_fold_train, y_fold_val = y_temp.iloc[train_idx], y_temp.iloc[val_idx]
    
    # Train models
    print("Training LightGBM...")
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': 0.05,
        'num_leaves': 31,
        'max_depth': 6,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'random_state': 42,
        'verbose': -1
    }
    
    train_data = lgb.Dataset(X_fold_train, label=y_fold_train)
    val_data = lgb.Dataset(X_fold_val, label=y_fold_val, reference=train_data)
    
    callbacks = [lgb.early_stopping(stopping_rounds=100, verbose=False)]
    
    model_lgb = lgb.train(
        params,
        train_data,
        valid_sets=[val_data],
        num_boost_round=1000,
        callbacks=callbacks
    )
    
    print("Training XGBoost...")
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'learning_rate': 0.05,
        'max_depth': 6,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'n_estimators': 1000,
        'random_state': 42,
        'verbosity': 0
    }
    
    model_xgb = xgb.XGBRegressor(**params)
    model_xgb.fit(
        X_fold_train, y_fold_train,
        eval_set=[(X_fold_val, y_fold_val)],
        # early_stopping_rounds=100,
        verbose=False
    )
    
    print("Training CatBoost...")
    params = {
        'loss_function': 'RMSE',
        'learning_rate': 0.05,
        'depth': 6,
        'iterations': 1000,
        'random_seed': 42,
        'l2_leaf_reg': 3,
        'bootstrap_type': 'Bayesian',
        'verbose': False
    }
    
    model_cb = cb.CatBoost(params)
    model_cb.fit(
        X_fold_train, y_fold_train,
        eval_set=(X_fold_val, y_fold_val),
        early_stopping_rounds=100,
        verbose=False
    )

     # Make predictions on validation fold
    preds_lgb = model_lgb.predict(X_fold_val)
    preds_xgb = model_xgb.predict(X_fold_val)
    preds_cb = model_cb.predict(X_fold_val)
    preds_blend = 0.4 * preds_lgb + 0.4 * preds_xgb + 0.2 * preds_cb

    # Experiment clipping
    min_bpm = max(60, y_fold_train.min())  # Most songs have at least 60 BPM
    max_bpm = min(200, y_fold_train.max())  # Most songs have at most 200 BPM
    preds_lgb_clipped = np.clip(preds_lgb, min_bpm, max_bpm)

    # Store out-of-fold predictions
    oof_predictions[val_idx] = preds_blend

    # Calculate and display fold metrics
    rmse_lgb = root_mean_squared_error(y_fold_val, preds_lgb)
    rmse_xgb = root_mean_squared_error(y_fold_val, preds_xgb)
    rmse_cb = root_mean_squared_error(y_fold_val, preds_cb)
    rmse_blend = root_mean_squared_error(y_fold_val, preds_blend)
    rmse_lgb_clipped = root_mean_squared_error(y_fold_val, preds_lgb_clipped)

    print(f"Fold {fold_idx + 1} Results:")
    print(f"LightGBM RMSE: {rmse_lgb:.5f}")
    print(f"XGBoost RMSE: {rmse_xgb:.5f}")
    print(f"CatBoost RMSE: {rmse_cb:.5f}")
    print(f"Blended RMSE: {rmse_blend:.5f}")
    print(f"LightGBM Clipped RMSE: {rmse_lgb_clipped:.5f}")

    rmse_list_lgb.append(rmse_lgb)
    rmse_list_xgb.append(rmse_xgb)
    rmse_list_cb.append(rmse_cb)
    rmse_list_blend.append(rmse_blend)
    rmse_list_lgb_clipped.append(rmse_lgb_clipped)

    # Store models for this fold
    models.append({
        'fold': fold_idx,
        'model_lgb': model_lgb,
        'model_xgb': model_xgb,
        'model_cb': model_cb,
    })

cv_score = root_mean_squared_error(y_temp, oof_predictions)

print("-"*100)
print(f"Average RMSE Results:")
print(f"LightGBM RMSE: {np.mean(rmse_list_lgb):.5f}")
print(f"XGBoost RMSE: {np.mean(rmse_list_xgb):.5f}")
print(f"CatBoost RMSE: {np.mean(rmse_list_cb):.5f}")
print(f"Blended RMSE: {np.mean(rmse_list_blend):.5f}")
print(f"LightGBM Clipped RMSE: {np.mean(rmse_list_lgb_clipped):.5f}")


Training fold 1/5
Training LightGBM...
Training XGBoost...
Training CatBoost...
Fold 1 Results:
LightGBM RMSE: 26.43883
XGBoost RMSE: 26.51528
CatBoost RMSE: 26.43946
Blended RMSE: 26.45103
LightGBM Clipped RMSE: 26.43883

Training fold 2/5
Training LightGBM...
Training XGBoost...
Training CatBoost...
Fold 2 Results:
LightGBM RMSE: 26.48372
XGBoost RMSE: 26.56617
CatBoost RMSE: 26.48444
Blended RMSE: 26.49808
LightGBM Clipped RMSE: 26.48372

Training fold 3/5
Training LightGBM...
Training XGBoost...
Training CatBoost...
Fold 3 Results:
LightGBM RMSE: 26.52298
XGBoost RMSE: 26.59748
CatBoost RMSE: 26.52359
Blended RMSE: 26.53414
LightGBM Clipped RMSE: 26.52298

Training fold 4/5
Training LightGBM...
Training XGBoost...
Training CatBoost...
Fold 4 Results:
LightGBM RMSE: 26.44308
XGBoost RMSE: 26.51526
CatBoost RMSE: 26.44303
Blended RMSE: 26.45407
LightGBM Clipped RMSE: 26.44308

Training fold 5/5
Training LightGBM...
Training XGBoost...
Training CatBoost...
Fold 5 Results:
LightGBM RM

Advanced Stacking Ensemble

In [12]:
# Build an advanced stacking ensemble
print("\nBuilding advanced stacking ensemble...")

# Filter models if any failed to train
valid_models = []
for model in models:
    if all(m is not None for m in [model['model_lgb'], model['model_xgb'], model['model_cb']]):
        valid_models.append(model)

if len(valid_models) > 0:
    # Create meta-features for stacking
    X_meta_train = np.column_stack([
        np.array([model['model_lgb'].predict(X_temp) for model in valid_models]).mean(axis=0),
        np.array([model['model_xgb'].predict(X_temp) for model in valid_models]).mean(axis=0),
        np.array([model['model_cb'].predict(X_temp) for model in valid_models]).mean(axis=0)
    ])

    # # Create meta-features for test set
    # X_meta_test = np.column_stack([
    #     np.array([model['model_lgb'].predict(X_test) for model in valid_models]).mean(axis=0),
    #     np.array([model['model_xgb'].predict(X_test) for model in valid_models]).mean(axis=0),
    #     np.array([model['model_cb'].predict(X_test) for model in valid_models]).mean(axis=0)
    # ])

    # Train a Ridge meta-model
    meta_model = Ridge(alpha=1.0)
    meta_model.fit(X_meta_train, y_temp)

    # # Make final predictions
    # stacking_predictions = meta_model.predict(X_meta_test)

    # Analyze the performance of the stacking model
    stacking_oof_preds = meta_model.predict(X_meta_train)
    stacking_cv_score = root_mean_squared_error(y_temp, stacking_oof_preds)
    print(f"Stacking Ensemble CV RMSE: {stacking_cv_score:.5f}")

    # Compare with the simple average approach
    print(f"Simple Average Ensemble CV RMSE: {cv_score:.5f}")

    # # Select the better performing approach for final predictions
    # if stacking_cv_score < cv_score:
    #     print("Using stacking ensemble for final predictions")
    #     final_predictions = stacking_predictions
    # else:
    #     print("Using simple average ensemble for final predictions")
    #     final_predictions = test_predictions
else:
    print("Not enough valid models for stacking. Using simple average ensemble.")
    # final_predictions = test_predictions


Building advanced stacking ensemble...
Stacking Ensemble CV RMSE: 23.57104
Simple Average Ensemble CV RMSE: 26.47223


Compiled comparison result

In [13]:
result_dict  ={
    "Original" : rmse_ori,
    "Feature Engineering" : rmse_fe,
    "Robust Scaler" : rmse_scaled,
    "LightGBM" : np.mean(rmse_list_lgb),
    "XGBoost" : np.mean(rmse_list_xgb),
    "CatBoost" : np.mean(rmse_list_cb),
    "Blended" : np.mean(rmse_list_blend),
    "LightGBM Clipped" : np.mean(rmse_list_lgb_clipped),
}
df_result = pd.DataFrame([result_dict]).T
df_result.columns = ['rmse']
df_result.sort_values('rmse', ascending=True)

,rmse
LightGBM,26.459208
LightGBM Clipped,26.459208
CatBoost,26.459783
Robust Scaler,26.466298
Original,26.466469
Feature Engineering,26.466643
Blended,26.472199
XGBoost,26.537348


Learnings :

- Better to use robust scaler rather than outlier removal
- Experiment using stronger models such as LightGBM/XGBoost/CatBoost
- Blended and stacking models can improve predictions